In [24]:
import re
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from rich import print
from rich.table import Table
from rich.console import Console

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
terminal_width = shutil.get_terminal_size().columns
df = pd.read_csv("labeled_customer_data.csv")
print(df.head(10))

Customer ID  Age  Gender Loyalty Member Product Type      SKU  Rating  \
0         1000   53    Male             No   Smartphone  SKU1004       2   
1         1000   53    Male             No       Tablet  SKU1002       3   
2         1002   41    Male             No       Laptop  SKU1005       3   
3         1002   41    Male            Yes   Smartphone  SKU1004       2   
4         1003   75    Male            Yes   Smartphone  SKU1001       5   
5         1004   41  Female             No   Smartphone  SKU1001       5   
6         1005   25  Female             No   Smartwatch  SKU1003       3   
7         1005   25  Female             No       Laptop  SKU1005       3   
8         1006   24    Male             No   Smartphone  SKU1004       2   
9         1006   24    Male            Yes       Laptop  SKU1005       3   

  Order Status Payment Method  Total Price  Unit Price  Quantity  \
0    Cancelled    Credit Card      5538.33      791.19         7   
1    Completed         Paypal       741.09      247.03         3   
2    Completed    Credit Card      1855.84      463.96         4   
3    Completed           Cash      3164.76      791.19         4   
4    Completed           Cash        41.50       20.75         2   
5    Completed    Credit Card        83.00       20.75         4   
6    Completed         Paypal      7603.47      844.83         9   
7    Completed     Debit Card      4175.64      463.96         9   
8    Cancelled     Debit Card      5538.33      791.19         7   
9    Completed           Cash      4175.64      463.96         9   

  Purchase Date Shipping Type                    Add-ons Purchased  \
0    2024-03-20      Standard        Accessory,Accessory,Accessory   
1    2024-04-20     Overnight                         Impulse Item   
2    2023-10-17       Express                                  NaN   
3    2024-08-09     Overnight            Impulse Item,Impulse Item   
4    2024-05-21       Express                            Accessory   
5    2024-05-26      Standard               Impulse Item,Accessory   
6    2024-01-30     Overnight                                  NaN   
7    2024-06-24     Overnight  Extended Warranty,Extended Warranty   
8    2023-10-03      Standard                         Impulse Item   
9    2024-01-01       Express                                  NaN   

   Add-on Total            Age_Group         Spending_Category  \
0         40.21   45-59 (Pre Senior)   High Spender ($5k-$10k)   
1         26.09   45-59 (Pre Senior)  Medium Spender (100-$5k)   
2          0.00   31-44 (Middle Age)  Medium Spender (100-$5k)   
3         60.16   31-44 (Middle Age)  Medium Spender (100-$5k)   
4         35.56         60+ (Senior)      Low Spender (0-$500)   
5         65.78   31-44 (Middle Age)      Low Spender (0-$500)   
6          0.00  18-30 (Young Adult)   High Spender ($5k-$10k)   
7         75.33  18-30 (Young Adult)  Medium Spender (100-$5k)   
8         43.05  18-30 (Young Adult)   High Spender ($5k-$10k)   
9          0.00  18-30 (Young Adult)  Medium Spender (100-$5k)   

         Item_Price_Category           Addons_Spending_Category  
0    Premium Item ($500-$1k)        Low Add-ons Spender (0-$50)  
1  Standard Item ($100-$500)        Low Add-ons Spender (0-$50)  
2  Standard Item ($100-$500)        Low Add-ons Spender (0-$50)  
3    Premium Item ($500-$1k)  Medium Add-ons Spender ($51-$100)  
4       Budget Item (0-$100)        Low Add-ons Spender (0-$50)  
5       Budget Item (0-$100)  Medium Add-ons Spender ($51-$100)  
6    Premium Item ($500-$1k)        Low Add-ons Spender (0-$50)  
7  Standard Item ($100-$500)  Medium Add-ons Spender ($51-$100)  
8    Premium Item ($500-$1k)        Low Add-ons Spender (0-$50)  
9  Standard Item ($100-$500)        Low Add-ons Spender (0-$50)

In [ ]:
# Tabulating Data
# 1. Tabel Tabulasi Product Type
summary_df = (
    df.groupby("Product Type")
    .agg(
        Total_Revenue=("Total Price", "sum"),
        Average_Rating=("Rating", "mean"),
        Average_Quantity=("Quantity", "mean"),
        Buyer_Age=("Age", "mean"),
        Average_Addon_Total=("Add-on Total", "mean"),
        Transaction_Count=("Product Type", "count"),
    )
    .reset_index()
)

# Formatting
summary_df["Total_Revenue"] = summary_df["Total_Revenue"].apply(
    lambda x: f"${x/1000:,.0f}k"
)
summary_df["Average_Rating"] = summary_df["Average_Rating"].round(2)
summary_df["Average_Quantity"] = summary_df["Average_Quantity"].round(2)
summary_df["Buyer_Age"] = summary_df["Buyer_Age"].round(1)
summary_df["Average_Addon_Total"] = summary_df["Average_Addon_Total"].apply(
    lambda x: f"${x:.2f}"
)

table = Table(
    title="Tabulasi Utama Berdasarkan Product Type",
    title_style="bold yellow",
    show_lines=True,
)
for column in summary_df.columns:
    table.add_column(column, style="white bold", justify="center", no_wrap=True)
for _, row in summary_df.iterrows():
    table.add_row(*[str(item) for item in row])
console = Console()
console.print(table)

# 2. Tabel Distribusi Frekuensi Product Type terhadap Age Group
age_product_table = pd.crosstab(df["Product Type"], df["Age_Group"])
print("\nDistribusi Frekuensi Product Type terhadap Age Group")
print(age_product_table)

# 3. Tabel Distribusi Frekuensi Product Type terhadap Shipping Type
shipping_product_table = pd.crosstab(df["Age_Group"], df["Payment Method"])
print("\nDistribusi Frekuensi Gender terhadap Payment Method")
print(shipping_product_table)

# 4. Tabel Distribusi Frekuensi Product Type terhadap Kelompok Umur Perempuan dan Laki-laki
filtered_female_df = df[(df["Gender"] == "Female") & (df["Age_Group"])]
filtered_male_df = df[(df["Gender"] == "Male") & (df["Age_Group"])]
female_shipping = pd.crosstab(
    index=filtered_female_df["Age_Group"], columns=filtered_female_df["Product Type"]
)
male_product = pd.crosstab(
    index=filtered_male_df["Age_Group"], columns=filtered_male_df["Product Type"]
)
print("Distribusi Frekuensi Product Type untuk Kelompok Usia Perempuan:")
print(female_shipping)
print("Distribusi Frekuensi Product Type untuk Kelompok Usia Laki-laki:")
print(male_product)

                                      Tabulasi Utama Berdasarkan Product Type                                      
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Product Ty… ┃ Total_Reven… ┃ Average_Rati… ┃ Average_Quanti… ┃ Buyer_A… ┃ Average_Addon_Tot… ┃ Transaction_Cou… ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Headphones  │   $4,041k    │     2.99      │      5.56       │   48.2   │       $82.64       │       2011       │
├─────────────┼──────────────┼───────────────┼─────────────────┼──────────┼────────────────────┼──────────────────┤
│   Laptop    │   $12,296k   │     2.98      │      5.43       │   48.8   │       $62.93       │       3973       │
├─────────────┼──────────────┼───────────────┼─────────────────┼──────────┼────────────────────┼──────────────────┤
│ Smartphone  │   $21,517k   │     3.32      │      5.46       │   49.1   │       $55.37       │       5978       │
├─────────────┼──────────────┼───────────────┼─────────────────┼──────────┼────────────────────┼──────────────────┤
│ Smartwatch  │   $14,036k   │     2.99      │       5.5       │   49.3   │       $61.37       │       3934       │
├─────────────┼──────────────┼───────────────┼─────────────────┼──────────┼────────────────────┼──────────────────┤
│   Tablet    │   $11,712k   │     3.02      │      5.52       │   49.1   │       $62.44       │       4104       │
└─────────────┴──────────────┴───────────────┴─────────────────┴──────────┴────────────────────┴──────────────────┘

Distribusi Frekuensi Product Type terhadap Age Group

Age_Group     18-30 (Young Adult)  31-44 (Middle Age)  45-59 (Pre Senior)  \
Product Type                                                                
Headphones                    440                 462                 469   
Laptop                        811                 916                 971   
Smartphone                   1176                1355                1464   
Smartwatch                    797                 888                 899   
Tablet                        845                 924                 939   

Age_Group     60+ (Senior)  
Product Type                
Headphones             640  
Laptop                1275  
Smartphone            1983  
Smartwatch            1350  
Tablet                1396

Distribusi Frekuensi Gender terhadap Payment Method

Payment Method       Bank Transfer  Cash  Credit Card  Debit Card  PayPal  \
Age_Group                                                                   
18-30 (Young Adult)            676   483         1219         498     689   
31-44 (Middle Age)             776   561         1280         601     780   
45-59 (Pre Senior)             820   602         1398         556     757   
60+ (Senior)                  1099   846         1971         816    1058   

Payment Method       Paypal  
Age_Group                    
18-30 (Young Adult)     504  
31-44 (Middle Age)      547  
45-59 (Pre Senior)      609  
60+ (Senior)            854

Distribusi Frekuensi Shipping Type untuk Female Young Adults:

Product Type         Headphones  Laptop  Smartphone  Smartwatch  Tablet
Age_Group                                                              
18-30 (Young Adult)         207     420         606         400     413
31-44 (Middle Age)          211     444         708         441     468
45-59 (Pre Senior)          241     469         685         416     461
60+ (Senior)                308     638         981         644     674

Product Type         Headphones  Laptop  Smartphone  Smartwatch  Tablet
Age_Group                                                              
18-30 (Young Adult)         233     390         570         397     432
31-44 (Middle Age)          251     472         647         447     456
45-59 (Pre Senior)          228     502         779         483     478
60+ (Senior)                332     637        1002         706     722